In [196]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [197]:
loan_train = pd.read_csv(r"D:\self learning\loan_data_train.csv")

In [198]:
loan_train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [199]:
loan_train.count()

Loan_ID              614
Gender               601
Married              611
Dependents           599
Education            614
Self_Employed        582
ApplicantIncome      614
CoapplicantIncome    614
LoanAmount           592
Loan_Amount_Term     600
Credit_History       564
Property_Area        614
Loan_Status          614
dtype: int64

In [200]:
# Counting number of males and females

male = 0
female = 0

for row in loan_train['Gender']:
    if row == 'Male':
        male+= 1
    else:
        female+= 1

print("Male count is " + str(male))
print("Female count is " + str(female))

Male count is 489
Female count is 125


In [201]:
# Since the number of males are too much in proportion therefore replacing the nulls with Male

loan_train['Gender'].fillna('Male' , inplace = True)

In [202]:
# Counting number of married and unmarried

married = 0
unmarried = 0

for row in loan_train['Married']:
    if row == 'Yes':
        married+=1
    else:
        unmarried+=1

print("Married count is "+ str(married))
print("Unmarried count is "+ str(unmarried))

Married count is 398
Unmarried count is 216


In [203]:
# Since only 3 of the married status is unkonwn we would replace that by the mode value

loan_train['Married'].fillna(loan_train['Married'].mode()[0] , inplace = True)


In [204]:
loan_train['Loan_Amount_Term'].value_counts()

360.0    512
180.0     44
480.0     15
300.0     13
84.0       4
240.0      4
120.0      3
36.0       2
60.0       2
12.0       1
Name: Loan_Amount_Term, dtype: int64

In [205]:
# 360 Months is the clear winner , so replace the Nan with the mode 

loan_train['Loan_Amount_Term'].fillna(loan_train['Loan_Amount_Term'].mode()[0] , inplace = True)

In [206]:


from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

In [207]:
# Since Dependents is a big constraint seen by the banks before providing the loan , we would predict that value

X_data = loan_train[['Gender' , 'Married' , 'Education','ApplicantIncome' , 'CoapplicantIncome','Property_Area']]
y_data = loan_train[['Dependents']]

# Replacing values in the dataset
X_data['Gender'] = pd.get_dummies(X_data['Gender'] , drop_first=True)
X_data['Married'] = pd.get_dummies(X_data['Married'] , drop_first=True)
X_data['Education'] = pd.get_dummies(X_data['Education'] , drop_first=True)
X_data['Property_Area'] = pd.get_dummies(X_data['Property_Area'] , drop_first=True)

X_train = X_data[loan_train.Dependents.notnull()]

y_train = y_data[loan_train.Dependents.notnull()]

X_test = X_data[loan_train.Dependents.isnull()]

classifier_dep = LinearSVC()
classifier_dep.fit(X_train , y_train)
y_test = classifier_dep.predict(X_test)

# Replacing the Null values with the predicted values

loan_train.loc[pd.isnull(loan_train['Dependents']) , 'Dependents'] = y_test


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  i

In [208]:
loan_train.count()

Loan_ID              614
Gender               614
Married              614
Dependents           614
Education            614
Self_Employed        582
ApplicantIncome      614
CoapplicantIncome    614
LoanAmount           592
Loan_Amount_Term     614
Credit_History       564
Property_Area        614
Loan_Status          614
dtype: int64

In [209]:
# Since Self Employed is a big constraint seen by the banks before providing the loan , we would predict that value

# Replacing the value 3+ with 3 

loan_train.loc[loan_train['Dependents']=='3+' , 'Dependents'] = 3

train_data=loan_train[loan_train.Self_Employed.notnull()]

SE_X_train = train_data[['Gender','Married','Dependents','Education','ApplicantIncome']]
SE_y_train = train_data[['Self_Employed']]

SE_X_train['Gender'] = pd.get_dummies(SE_X_train['Gender'] , drop_first=True)
SE_X_train['Married'] = pd.get_dummies(SE_X_train['Married'] , drop_first=True)
SE_X_train['Education']=pd.get_dummies(SE_X_train['Education'] , drop_first=True)
SE_y_train['Self_Employed']=pd.get_dummies(SE_y_train['Self_Employed'] , drop_first=True)

predict_data = loan_train[loan_train.Self_Employed.isnull()]

SE_X_predict = predict_data[['Gender','Married','Dependents','Education','ApplicantIncome']]
SE_y_predict = predict_data[['Self_Employed']]

SE_X_predict['Gender'] = pd.get_dummies(SE_X_predict['Gender'] , drop_first=True)
SE_X_predict['Married'] = pd.get_dummies(SE_X_predict['Married'] , drop_first=True)
SE_X_predict['Education']=pd.get_dummies(SE_X_predict['Education'] , drop_first=True)


classifier_SE = LinearSVC()
classifier_SE.fit(SE_X_train , SE_y_train)
SE_predicted_value = classifier_SE.predict(SE_X_predict)

loan_train.loc[pd.isnull(loan_train['Self_Employed']) , 'Self_Employed'] = SE_predicted_value



D:\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

In [210]:
loan_train.count()

Loan_ID              614
Gender               614
Married              614
Dependents           614
Education            614
Self_Employed        614
ApplicantIncome      614
CoapplicantIncome    614
LoanAmount           592
Loan_Amount_Term     614
Credit_History       564
Property_Area        614
Loan_Status          614
dtype: int64

In [211]:
loan_train['Credit_History'].value_counts()

1.0    475
0.0     89
Name: Credit_History, dtype: int64

In [212]:
# Replacing Credit History with the mode value

loan_train['Credit_History'].fillna(loan_train['Credit_History'].mode()[0] , inplace = True)

In [213]:
loan_train.count()

Loan_ID              614
Gender               614
Married              614
Dependents           614
Education            614
Self_Employed        614
ApplicantIncome      614
CoapplicantIncome    614
LoanAmount           592
Loan_Amount_Term     614
Credit_History       614
Property_Area        614
Loan_Status          614
dtype: int64

In [214]:
# Filling up the Loan Amount does not make any sense as of now , so would be removing the rows with Nan

final_train = loan_train[loan_train.LoanAmount.notnull()]

In [215]:
# Label Encoding the data for the training set

final_train['Gender'] = pd.get_dummies(final_train['Gender'] , drop_first = True)
final_train['Married'] = pd.get_dummies(final_train['Married'] , drop_first = True)
final_train['Education'] = pd.get_dummies(final_train['Education'] , drop_first = True)
final_train['Self_Employed'] = pd.get_dummies(final_train['Self_Employed'] , drop_first = True)
final_train['Property_Area'] = pd.get_dummies(final_train['Property_Area'] , drop_first = True)
final_train['Loan_Status'] = pd.get_dummies(final_train['Loan_Status'] , drop_first = True)

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [216]:
final_train.dtypes

Loan_ID               object
Gender                 uint8
Married                uint8
Dependents            object
Education              uint8
Self_Employed          uint8
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area          uint8
Loan_Status            uint8
dtype: object

In [217]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix


In [218]:
X = final_train[['Gender','Married','Education','Self_Employed','ApplicantIncome','CoapplicantIncome',\
                 'LoanAmount','Loan_Amount_Term','Credit_History','Property_Area']]

y = final_train[['Loan_Status']]

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2)

classifier_NB = MultinomialNB()
classifier_LR = LogisticRegression()
classifier_SVM = LinearSVC()

classifier_NB.fit(X_train , y_train)
print("Confusion Matrix for Naive Bayes is")
print(confusion_matrix(y_test , classifier_NB.predict(X_test)))

classifier_LR.fit(X_train , y_train)
print("Confusion Matrix for Logistic Regression is")
print(confusion_matrix(y_test , classifier_LR.predict(X_test)))

classifier_SVM.fit(X_train , y_train)
print("Confusion Matrix for Support Vector Machine is")
print(confusion_matrix(y_test , classifier_SVM.predict(X_test)))



Confusion Matrix for Naive Bayes is
[[13 14]
 [52 40]]
Confusion Matrix for Logistic Regression is
[[11 16]
 [ 2 90]]
Confusion Matrix for Support Vector Machine is
[[ 0 27]
 [ 0 92]]


D:\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [219]:
# Logistic Regression comes out to be the winner , therefore we will train the model finally with Logistic Regression

classifier_LR.fit(X_train , y_train)

D:\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [247]:
loan_test = pd.read_csv(r"D:\self learning\loan_data_test.csv")

In [248]:
loan_test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [249]:
# Replacing the Nan values with the same way as we did for the training set

loan_test.count()

# Replacing Gender

loan_test['Gender'].fillna(loan_test['Gender'].mode()[0] , inplace = True)

# Replacing Credit History

loan_test['Credit_History'].fillna(loan_test['Credit_History'].mode()[0] , inplace = True)

# Replacing Laon Amount with the mean value

loan_test['LoanAmount'].fillna(loan_test['LoanAmount'].mean() , inplace = True)

# Replacing Loan Amount Term with the mode value

loan_test['Loan_Amount_Term'].fillna(loan_test['Loan_Amount_Term'].mode()[0] , inplace = True)





In [250]:
# Encoding the Test Data

loan_test['Gender'] = pd.get_dummies(loan_test['Gender'] , drop_first = True)
loan_test['Married'] = pd.get_dummies(loan_test['Married'] , drop_first = True)
loan_test['Education'] = pd.get_dummies(loan_test['Education'] , drop_first = True)
loan_test['Property_Area'] = pd.get_dummies(loan_test['Property_Area'] , drop_first = True)




In [253]:
loan_test.count()

Loan_ID              367
Gender               367
Married              367
Dependents           367
Education            367
Self_Employed        344
ApplicantIncome      367
CoapplicantIncome    367
LoanAmount           367
Loan_Amount_Term     367
Credit_History       367
Property_Area        367
dtype: int64

In [252]:
# Filling the Nan Values of dependents by using the training classfier

final_loan_dep = loan_test[pd.isnull(loan_test['Dependents'])]

X_loan_test_dep = final_loan_dep[['Gender' , 'Married' , 'Education','ApplicantIncome' , 'CoapplicantIncome','Property_Area']]

dep_replace = classifier_dep.predict(X_loan_test_dep)

loan_test.loc[pd.isnull(loan_test['Dependents']) , 'Dependents'] = dep_replace

In [268]:
# Filling the Nan values of Self Employed with the training classifier

# Replacing the value of 3+ in dependents to 3

loan_test.loc[loan_test['Dependents']=='3+' , 'Dependents'] = 3  

# Running the predict function to predict the Nan values of Self Employed

final_loan_se = loan_test[loan_test['Self_Employed'].isnull()]

X_loan_test_se = final_loan_se[['Gender','Married','Dependents','Education','ApplicantIncome']]

se_replace = classifier_SE.predict(X_loan_test_se)

loan_test.loc[pd.isnull(loan_test['Self_Employed']) , 'Self_Employed'] = se_replace

# Label Encoding the Not null values of SElf Employed

loan_test['Self_Employed'] = pd.get_dummies(loan_test['Self_Employed'] , drop_first = True)

In [276]:
loan_test.Self_Employed.value_counts()

0    345
1     22
Name: Self_Employed, dtype: int64

In [277]:
loan_test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,1,1,0,0,0,5720,0,110.0,360.0,1.0,0
1,LP001022,1,1,1,0,0,3076,1500,126.0,360.0,1.0,0
2,LP001031,1,1,2,0,0,5000,1800,208.0,360.0,1.0,0
3,LP001035,1,1,2,0,0,2340,2546,100.0,360.0,1.0,0
4,LP001051,1,0,0,1,0,3276,0,78.0,360.0,1.0,0


In [278]:
# Predicting the test output

test_input_param = loan_test[['Gender','Married','Education','Self_Employed','ApplicantIncome','CoapplicantIncome',\
                 'LoanAmount','Loan_Amount_Term','Credit_History','Property_Area']]

loan_output = classifier_LR.predict(test_input_param)
        

In [281]:
loan_test['Loan_Status'] = loan_output

In [282]:
loan_test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001015,1,1,0,0,0,5720,0,110.0,360.0,1.0,0,1
1,LP001022,1,1,1,0,0,3076,1500,126.0,360.0,1.0,0,1
2,LP001031,1,1,2,0,0,5000,1800,208.0,360.0,1.0,0,1
3,LP001035,1,1,2,0,0,2340,2546,100.0,360.0,1.0,0,1
4,LP001051,1,0,0,1,0,3276,0,78.0,360.0,1.0,0,1
